# Incremental deep learning with river & tf keras
## Imports

In [1]:
from pathlib import Path

from river import preprocessing, tree, evaluate, datasets, compose
import river
from IncrementalDL.OnlineKeras.classifier import Keras2RiverClassifier
from tensorflow.python.keras.losses import categorical_crossentropy

In [2]:
n_samples = 1_000
seed = 42

In [3]:
dataset = datasets.synth.RandomRBF(seed_model=7, seed_sample=seed,n_classes=10,n_features=200).take(n_samples)
metric = river.metrics.ROCAUC()

In [4]:
from tensorflow.python.keras.optimizer_v2.adam import Adam
from tensorflow.python.keras.layers import Dense
from tensorflow.python.keras import Sequential
from tensorflow.python.keras.backend import binary_crossentropy

def build_fn(n_layers):
    keras_model = Sequential()
    keras_model.add(Dense(5,activation='relu'))
    keras_model.add(Dense(1, activation='softmax'))
    optimizer = Adam(learning_rate=1e-3)
    keras_model.compile(loss=binary_crossentropy, optimizer=optimizer, metrics=[])
    return keras_model

In [5]:
model = compose.Pipeline(
    preprocessing.StandardScaler(),
    Keras2RiverClassifier(
        build_fn=build_fn,
        batch_size=1,
        seed=seed,
        n_layers = 5,
    )
)

2021-08-23 13:29:16.214348: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [6]:
for x, y in dataset:
    y_pred = model.predict_proba_one(x)
    model.learn_one(x, y)
    metric.update(y, y_pred)
metric

ROCAUC: 0.690047